<a href="https://colab.research.google.com/github/Albly/PageRank/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Albly/PageRank/blob/master/main.ipynb)

In [ ]:
import numpy as np
import plotly.figure_factory as ff
import seaborn as sns
import os
import sys
from pathlib import Path
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm, trange


git_root = !git rev-parse --show-toplevel
already_in_repo = os.path.exists(git_root[0])

if not already_in_repo:
    !git clone https://github.com/Albly/PageRank $repo_dir


from PageRank.PageRankSolver.pretty_print import *
import PageRank.PageRankSolver.fast.solver as fast_solver
import PageRank.PageRankSolver.fast.tmatrix as fast_tmatrix
import PageRank.PageRankSolver.slow.solver as solver
import PageRank.PageRankSolver.slow.tmatrix as tmatrix


%load_ext autoreload
%autoreload 2

Cloning into 'PageRank'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 104 (delta 39), reused 91 (delta 26), pack-reused 0
Receiving objects: 100% (104/104), 450.40 KiB | 2.00 MiB/s, done.
Resolving deltas: 100% (39/39), done.


This package contatins two implementations: slow - use usual python syntax. fast - jitted python implementation. All the experiments are done with "fast" mode. However both modes have the same functions. 

# **Instruments for visualisation**

In [ ]:
# Create matrix using Erdős–Rényi model
A = fast_tmatrix.generate(size = 10, Pr = 0.8)
# Function for matrix plotting powered by Plotly
plot_matrix(A, width= 300, height=300)

In [ ]:
# Function for matrix printing
matprint(A, '.2f')

0.00  0.17  0.12  0.12  0.12  0.14  0.12  0.00  0.11  0.12  
0.17  0.00  0.12  0.12  0.12  0.14  0.00  0.17  0.11  0.12  
0.17  0.00  0.00  0.12  0.12  0.00  0.12  0.17  0.11  0.12  
0.00  0.17  0.12  0.00  0.12  0.14  0.12  0.00  0.11  0.12  
0.17  0.17  0.12  0.12  0.00  0.14  0.12  0.17  0.11  0.12  
0.00  0.17  0.12  0.12  0.12  0.00  0.12  0.17  0.11  0.12  
0.17  0.17  0.12  0.12  0.12  0.00  0.00  0.17  0.11  0.00  
0.00  0.00  0.00  0.00  0.12  0.14  0.12  0.00  0.11  0.12  
0.17  0.17  0.12  0.12  0.12  0.14  0.12  0.17  0.00  0.12  
0.17  0.00  0.12  0.12  0.00  0.14  0.12  0.00  0.11  0.00  


# **Generators**

Model of the Erdős–Rényi is not the only one here, it has several modification to get special kinds of graphs with several problems: 

*   generate_dangled - where random column contains zeros
*   generate_segmented - where two parts of the graphs are disconnected

Also the Bollobás–Riordan generator was created:

*   generate_bollobas_riordan







In [ ]:
# dangled matrix
A = fast_tmatrix.generate_dangled(10,0.8)
plot_matrix(A)

In [ ]:
# segmented matrix
A = fast_tmatrix.generate_segmented(10,0.8)
plot_matrix(A)

In [ ]:
# Bollobas- Riordan model 
A = fast_tmatrix.generate_bollobas_riordan(n = 10, m = 10)
plot_matrix(A)

# **Matrix corrections**

Also for problematic cases were added two methods for matrix corrections:

*   add_weak_links - use such correstion with dampling factor: $$A' = (1 - \alpha)A + \alpha R $$ where R is a matrix of ones divided to matrix size 
*   connect_dangled_node - finds column with zeros and randomly connect the dangled node. WARNING! This function doesn't correct the linear dependent columns



In [ ]:
# create dangled node
A = fast_tmatrix.generate_dangled(size = 10, Pr = 0.7)
plot_matrix(A)
# fix it
A = fast_tmatrix.connect_dangled_node(A)
plot_matrix(A)

In [ ]:
# create segmented matrix
A = fast_tmatrix.generate_segmented(size = 10, Pr = 0.7)
plot_matrix(A)
# fix it
A = fast_tmatrix.add_weak_links(A)
plot_matrix(A)

# **Solvers**

Here 3 solvers are implemented: 


*   eig - simply calculate eigenvector using numpy eig function 
*   power_method - uses Power method for solving 
*   mcmc - Markov Chain Monte Carlo method

for high dimentional matrices its better not to use eig function, because it requires a long time even jitted version



In [ ]:
# eig function
A = fast_tmatrix.generate(10,0.7)
matprint(fast_solver.eig(A).real, '.4f')

0.2479  
0.2966  
0.2520  
0.3019  
0.3337  
0.2628  
0.3480  
0.4016  
0.4079  
0.2583  


In [ ]:
# MCMC method
matprint(fast_solver.mcmc(n_trials = 10**6,A=A ).reshape(-1,1), '.4f')

0.2467  
0.2961  
0.2531  
0.3030  
0.3327  
0.2618  
0.3485  
0.4017  
0.4088  
0.2576  


In [ ]:
# Power method 
matprint(fast_solver.power(A = A, n_trials = 5).reshape(-1,1), '.4f')

0.2478  
0.2964  
0.2522  
0.3017  
0.3336  
0.2629  
0.3479  
0.4018  
0.4081  
0.2583  


# **Experiments**

In [ ]:
Prs = np.linspace(0.01, 0.99, 30)
sizes = np.linspace(5,50, 25).astype(np.int32)

repeats = 5000

def problem_experiment_1(Prs, sizes, repeats):

    results = np.zeros((5, len(sizes), len(Prs)))

    for pr_idx in trange(len(Prs)):
        for size_idx in range(len(sizes)):
            for _ in range(repeats):
                A = fast_tmatrix.generate(sizes[size_idx] ,Pr = Prs[pr_idx])
                problem_vec = fast_tmatrix.check_matrix(A)
                results[:,size_idx, pr_idx] +=  problem_vec

    return results


n_s =  np.linspace(5,50, 20).astype(np.int32)
m_s  = np.linspace(1,50, 20).astype(np.int32)

repeats = 5000

def problem_experiment_2(m_s, n_s, repeats):

    results = np.zeros((5, len(n_s), len(m_s)))

    for m in trange(len(m_s)):
        for n in range(len(n_s)):
            for _ in range(repeats):
                A = fast_tmatrix.generate_bollobas_riordan(n_s[n] ,m_s[m])
                A = fast_tmatrix.add_weak_links(A)
                problem_vec = fast_tmatrix.check_matrix(A)
                results[:,n, m] +=  problem_vec

    return results




In [ ]:
res = problem_experiment_2(m_s, n_s, repeats)
np.savez('exp3', m_s = m_s, n_s= n_s, res = res)
files.download('exp3.npz')

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
res = problem_experiment_2(Prs, sizes, repeats)
np.savez('exp2', Prs = Prs, sizes= sizes, res = res)
files.download('exp2.npz')

  0%|          | 0/30 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download('sample_data/anscombe.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>